# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), May 6, 2020**

In [1]:
using DataFrames

## Manipulating columns of a `DataFrame`

### Renaming columns

Let's start with a `DataFrame` of `Bool`s that has default column names.

In [2]:
x = DataFrame(rand(Bool, 3, 4))

,x1,x2,x3,x4
,Bool,Bool,Bool,Bool
1,0,0,1,1
2,1,0,1,1
3,1,1,1,0


With `rename`, we create new `DataFrame`; here we rename the column `:x1` to `:A`. (`rename` also accepts collections of Pairs.)

In [3]:
rename(x, :x1 => :A)

,A,x2,x3,x4
,Bool,Bool,Bool,Bool
1,0,0,1,1
2,1,0,1,1
3,1,1,1,0


With `rename!` we do an in place transformation. 

This time we've applied a function to every column name (note that the function gets a column names as a string).

In [4]:
rename!(c -> c^2, x)

,x1x1,x2x2,x3x3,x4x4
,Bool,Bool,Bool,Bool
1,0,0,1,1
2,1,0,1,1
3,1,1,1,0


We can also change the name of a particular column without knowing the original.

Here we change the name of the third column, creating a new `DataFrame`.

In [5]:
rename(x, 3 => :third)

,x1x1,x2x2,third,x4x4
,Bool,Bool,Bool,Bool
1,0,0,1,1
2,1,0,1,1
3,1,1,1,0


If we pass a vector of names to `rename!`, we can change the names of all variables.

In [6]:
rename!(x, [:a, :b, :c, :d])

,a,b,c,d
,Bool,Bool,Bool,Bool
1,0,0,1,1
2,1,0,1,1
3,1,1,1,0


In all the above examples you could have used strings instead of symbols, e.g.

In [7]:
rename!(x, string.('a':'d'))

,a,b,c,d
,Bool,Bool,Bool,Bool
1,0,0,1,1
2,1,0,1,1
3,1,1,1,0


`rename!` allows for circular renaming of columns, e.g.:

In [8]:
x

,a,b,c,d
,Bool,Bool,Bool,Bool
1,0,0,1,1
2,1,0,1,1
3,1,1,1,0


In [9]:
rename!(x, "a"=>"d", "d"=>"a")

,d,b,c,a
,Bool,Bool,Bool,Bool
1,0,0,1,1
2,1,0,1,1
3,1,1,1,0


We get an error when we try to provide duplicate names

In [10]:
rename(x, fill(:a, 4))

ArgumentError: ArgumentError: Duplicate variable names: :a. Pass makeunique=true to make them unique using a suffix automatically.

 unless we pass `makeunique=true`, which allows us to handle duplicates in passed names.

In [11]:
rename(x, fill(:a, 4), makeunique=true)

,a,a_1,a_2,a_3
,Bool,Bool,Bool,Bool
1,0,0,1,1
2,1,0,1,1
3,1,1,1,0


### Reordering columns

We can reorder the `names(x)` vector as needed, creating a new `DataFrame`.

In [12]:
using Random
Random.seed!(1234)
x[:, shuffle(names(x))]

,b,a,c,d
,Bool,Bool,Bool,Bool
1,0,1,1,0
2,0,1,1,1
3,1,0,1,1


Also `select!` can be used to achieve this in place (or `select` to perform a copy):

In [13]:
x

,d,b,c,a
,Bool,Bool,Bool,Bool
1,0,0,1,1
2,1,0,1,1
3,1,1,1,0


In [14]:
select!(x, 4:-1:1);
x

,a,c,b,d
,Bool,Bool,Bool,Bool
1,1,1,0,0
2,1,1,0,1
3,0,1,1,1


### Merging/adding columns

In [15]:
x = DataFrame([(i,j) for i in 1:3, j in 1:4])

,x1,x2,x3,x4
,Tuple…,Tuple…,Tuple…,Tuple…
1,"(1, 1)","(1, 2)","(1, 3)","(1, 4)"
2,"(2, 1)","(2, 2)","(2, 3)","(2, 4)"
3,"(3, 1)","(3, 2)","(3, 3)","(3, 4)"


With `hcat` we can merge two `DataFrame`s. Also [x y] syntax is supported but only when DataFrames have unique column names.

In [16]:
hcat(x, x, makeunique=true)

,x1,x2,x3,x4,x1_1,x2_1,x3_1,x4_1
,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…
1,"(1, 1)","(1, 2)","(1, 3)","(1, 4)","(1, 1)","(1, 2)","(1, 3)","(1, 4)"
2,"(2, 1)","(2, 2)","(2, 3)","(2, 4)","(2, 1)","(2, 2)","(2, 3)","(2, 4)"
3,"(3, 1)","(3, 2)","(3, 3)","(3, 4)","(3, 1)","(3, 2)","(3, 3)","(3, 4)"


We can also use `hcat` to add a new column; a default name `:x1` will be used for this column, so `makeunique=true` is needed in our case.

In [17]:
y = hcat(x, [1,2,3], makeunique=true)

,x1,x2,x3,x4,x1_1
,Tuple…,Tuple…,Tuple…,Tuple…,Int64
1,"(1, 1)","(1, 2)","(1, 3)","(1, 4)",1
2,"(2, 1)","(2, 2)","(2, 3)","(2, 4)",2
3,"(3, 1)","(3, 2)","(3, 3)","(3, 4)",3


You can also prepend a vector with `hcat`.

In [18]:
hcat([1,2,3], x, makeunique=true)

,x1,x1_1,x2,x3,x4
,Int64,Tuple…,Tuple…,Tuple…,Tuple…
1,1,"(1, 1)","(1, 2)","(1, 3)","(1, 4)"
2,2,"(2, 1)","(2, 2)","(2, 3)","(2, 4)"
3,3,"(3, 1)","(3, 2)","(3, 3)","(3, 4)"


Alternatively you could append a vector with the following syntax. This is a bit more verbose but cleaner.

In [19]:
y = [x DataFrame(A=[1,2,3])]

,x1,x2,x3,x4,A
,Tuple…,Tuple…,Tuple…,Tuple…,Int64
1,"(1, 1)","(1, 2)","(1, 3)","(1, 4)",1
2,"(2, 1)","(2, 2)","(2, 3)","(2, 4)",2
3,"(3, 1)","(3, 2)","(3, 3)","(3, 4)",3


Here we do the same but add column `:A` to the front.

In [20]:
y = [DataFrame(A=[1,2,3]) x]

,A,x1,x2,x3,x4
,Int64,Tuple…,Tuple…,Tuple…,Tuple…
1,1,"(1, 1)","(1, 2)","(1, 3)","(1, 4)"
2,2,"(2, 1)","(2, 2)","(2, 3)","(2, 4)"
3,3,"(3, 1)","(3, 2)","(3, 3)","(3, 4)"


A column can also be added in the middle. Here a brute-force method is used and a new `DataFrame` is created.

In [21]:
using BenchmarkTools
@btime [$x[!, 1:2] DataFrame(A=[1,2,3]) $x[!, 3:4]]

  5.468 μs (107 allocations: 9.53 KiB)


,x1,x2,A,x3,x4
,Tuple…,Tuple…,Int64,Tuple…,Tuple…
1,"(1, 1)","(1, 2)",1,"(1, 3)","(1, 4)"
2,"(2, 1)","(2, 2)",2,"(2, 3)","(2, 4)"
3,"(3, 1)","(3, 2)",3,"(3, 3)","(3, 4)"


We could also do this with a specialized in place method `insertcols!`. Let's add `:newcol` to the `DataFrame` `y`.

In [22]:
insertcols!(y, 2, "newcol" => [1,2,3])

,A,newcol,x1,x2,x3,x4
,Int64,Int64,Tuple…,Tuple…,Tuple…,Tuple…
1,1,1,"(1, 1)","(1, 2)","(1, 3)","(1, 4)"
2,2,2,"(2, 1)","(2, 2)","(2, 3)","(2, 4)"
3,3,3,"(3, 1)","(3, 2)","(3, 3)","(3, 4)"


If you want to insert the same column name several times `makeunique=true` is needed as usual.

In [23]:
insertcols!(y, 2, :newcol => [1,2,3], makeunique=true)

,A,newcol_1,newcol,x1,x2,x3,x4
,Int64,Int64,Int64,Tuple…,Tuple…,Tuple…,Tuple…
1,1,1,1,"(1, 1)","(1, 2)","(1, 3)","(1, 4)"
2,2,2,2,"(2, 1)","(2, 2)","(2, 3)","(2, 4)"
3,3,3,3,"(3, 1)","(3, 2)","(3, 3)","(3, 4)"


We can see how much faster it is to insert a column with `insertcols!` than with `hcat` using `@btime` (note that we use here a `Pair` notation as an example).

In [24]:
@btime insertcols!(copy($x), 3, :A => [1,2,3])

  1.567 μs (31 allocations: 3.00 KiB)


,x1,x2,A,x3,x4
,Tuple…,Tuple…,Int64,Tuple…,Tuple…
1,"(1, 1)","(1, 2)",1,"(1, 3)","(1, 4)"
2,"(2, 1)","(2, 2)",2,"(2, 3)","(2, 4)"
3,"(3, 1)","(3, 2)",3,"(3, 3)","(3, 4)"


Let's use `insertcols!` to append a column in place (note that we dropped the index at which we insert the column)

In [25]:
insertcols!(x, :A => [1,2,3])

,x1,x2,x3,x4,A
,Tuple…,Tuple…,Tuple…,Tuple…,Int64
1,"(1, 1)","(1, 2)","(1, 3)","(1, 4)",1
2,"(2, 1)","(2, 2)","(2, 3)","(2, 4)",2
3,"(3, 1)","(3, 2)","(3, 3)","(3, 4)",3


and to in place prepend a column.

In [26]:
insertcols!(x, 1, :B => [1,2,3])

,B,x1,x2,x3,x4,A
,Int64,Tuple…,Tuple…,Tuple…,Tuple…,Int64
1,1,"(1, 1)","(1, 2)","(1, 3)","(1, 4)",1
2,2,"(2, 1)","(2, 2)","(2, 3)","(2, 4)",2
3,3,"(3, 1)","(3, 2)","(3, 3)","(3, 4)",3


Note that `insertcols!` can be used to insert several columns to a data frame at once and that it performs broadcasting if needed:

In [27]:
df = DataFrame(a = [1, 2, 3])

,a
,Int64
1,1
2,2
3,3


In [28]:
insertcols!(df, :b => "x", :c => 'a':'c', :d => Ref([1,2,3]))

,a,b,c,d
,Int64,String,Char,Array…
1,1,x,'a',"[1, 2, 3]"
2,2,x,'b',"[1, 2, 3]"
3,3,x,'c',"[1, 2, 3]"


Interestingly we can emulate `hcat` mutating the data frame in-place using `insertcols!`:

In [29]:
df1 = DataFrame(a=[1,2])

,a
,Int64
1,1
2,2


In [30]:
df2 = DataFrame(b=[2,3], c=[3,4])

,b,c
,Int64,Int64
1,2,3
2,3,4


In [31]:
hcat(df1, df2)

,a,b,c
,Int64,Int64,Int64
1,1,2,3
2,2,3,4


In [32]:
df1 # df1 is not touched

,a
,Int64
1,1
2,2


In [33]:
insertcols!(df1, pairs(eachcol(df2))...)

,a,b,c
,Int64,Int64,Int64
1,1,2,3
2,2,3,4


In [34]:
df1 # now we have changed df1

,a,b,c
,Int64,Int64,Int64
1,1,2,3
2,2,3,4


### Subsetting/removing columns

Let's create a new `DataFrame` `x` and show a few ways to create DataFrames with a subset of `x`'s columns.

In [35]:
x = DataFrame([(i,j) for i in 1:3, j in 1:5])

,x1,x2,x3,x4,x5
,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…
1,"(1, 1)","(1, 2)","(1, 3)","(1, 4)","(1, 5)"
2,"(2, 1)","(2, 2)","(2, 3)","(2, 4)","(2, 5)"
3,"(3, 1)","(3, 2)","(3, 3)","(3, 4)","(3, 5)"


First we could do this by index:

In [36]:
x[:, [1,2,4,5]] # use ! instead of : for non-copying operation

,x1,x2,x4,x5
,Tuple…,Tuple…,Tuple…,Tuple…
1,"(1, 1)","(1, 2)","(1, 4)","(1, 5)"
2,"(2, 1)","(2, 2)","(2, 4)","(2, 5)"
3,"(3, 1)","(3, 2)","(3, 4)","(3, 5)"


or by column name:

In [37]:
x[:, [:x1, :x4]]

,x1,x4
,Tuple…,Tuple…
1,"(1, 1)","(1, 4)"
2,"(2, 1)","(2, 4)"
3,"(3, 1)","(3, 4)"


We can also choose to keep or exclude columns by `Bool` (we need a vector whose length is the number of columns in the original `DataFrame`).

In [38]:
x[:, [true, false, true, false, true]]

,x1,x3,x5
,Tuple…,Tuple…,Tuple…
1,"(1, 1)","(1, 3)","(1, 5)"
2,"(2, 1)","(2, 3)","(2, 5)"
3,"(3, 1)","(3, 3)","(3, 5)"


Here we create a single column `DataFrame`,

In [39]:
x[:, [:x1]]

,x1
,Tuple…
1,"(1, 1)"
2,"(2, 1)"
3,"(3, 1)"


and here we access the vector contained in column `:x1`.

In [40]:
x[!, :x1] # use : instead of ! to copy

3-element Array{Tuple{Int64,Int64},1}:
 (1, 1)
 (2, 1)
 (3, 1)

In [41]:
x.x1 # the same

3-element Array{Tuple{Int64,Int64},1}:
 (1, 1)
 (2, 1)
 (3, 1)

We could grab the same vector by column number

In [42]:
x[!, 1]

3-element Array{Tuple{Int64,Int64},1}:
 (1, 1)
 (2, 1)
 (3, 1)

Note that getting a single column returns it without copying while creating a new `DataFrame` performs a copy of the column

In [43]:
x[!, 1] === x[!, [1]]

false

you can also use `Regex`, `All`, `Between` and `Not` from InvertedIndies.jl for column selection:

In [44]:
x[!, r"[12]"]

,x1,x2
,Tuple…,Tuple…
1,"(1, 1)","(1, 2)"
2,"(2, 1)","(2, 2)"
3,"(3, 1)","(3, 2)"


In [45]:
x[!, Not(1)]

,x2,x3,x4,x5
,Tuple…,Tuple…,Tuple…,Tuple…
1,"(1, 2)","(1, 3)","(1, 4)","(1, 5)"
2,"(2, 2)","(2, 3)","(2, 4)","(2, 5)"
3,"(3, 2)","(3, 3)","(3, 4)","(3, 5)"


In [46]:
x[!, Between(:x2, :x4)]

,x2,x3,x4
,Tuple…,Tuple…,Tuple…
1,"(1, 2)","(1, 3)","(1, 4)"
2,"(2, 2)","(2, 3)","(2, 4)"
3,"(3, 2)","(3, 3)","(3, 4)"


In [47]:
x[!, All(:x1, Between(:x3, :x5))]

,x1,x3,x4,x5
,Tuple…,Tuple…,Tuple…,Tuple…
1,"(1, 1)","(1, 3)","(1, 4)","(1, 5)"
2,"(2, 1)","(2, 3)","(2, 4)","(2, 5)"
3,"(3, 1)","(3, 3)","(3, 4)","(3, 5)"


In [48]:
select(x, :x1, Between(:x3, :x5), copycols=false) # the same as above

,x1,x3,x4,x5
,Tuple…,Tuple…,Tuple…,Tuple…
1,"(1, 1)","(1, 3)","(1, 4)","(1, 5)"
2,"(2, 1)","(2, 3)","(2, 4)","(2, 5)"
3,"(3, 1)","(3, 3)","(3, 4)","(3, 5)"


you can use `select` and `select!` functions to select a subset of columns from a data frame. `select` creates a new data frame and `select!` operates in place

In [49]:
df = copy(x)

,x1,x2,x3,x4,x5
,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…
1,"(1, 1)","(1, 2)","(1, 3)","(1, 4)","(1, 5)"
2,"(2, 1)","(2, 2)","(2, 3)","(2, 4)","(2, 5)"
3,"(3, 1)","(3, 2)","(3, 3)","(3, 4)","(3, 5)"


In [50]:
df2 = select(df, [1, 2])

,x1,x2
,Tuple…,Tuple…
1,"(1, 1)","(1, 2)"
2,"(2, 1)","(2, 2)"
3,"(3, 1)","(3, 2)"


In [51]:
select(df, Not([1, 2]))

,x3,x4,x5
,Tuple…,Tuple…,Tuple…
1,"(1, 3)","(1, 4)","(1, 5)"
2,"(2, 3)","(2, 4)","(2, 5)"
3,"(3, 3)","(3, 4)","(3, 5)"


by default `select` copies columns

In [52]:
df2[!, 1] === df[!, 1]

false

this can be avoided by using `copycols=false` keyword argument

In [53]:
df2 = select(df, [1, 2], copycols=false)

,x1,x2
,Tuple…,Tuple…
1,"(1, 1)","(1, 2)"
2,"(2, 1)","(2, 2)"
3,"(3, 1)","(3, 2)"


In [54]:
df2[!, 1] === df[!, 1]

true

using `select!` will modify the source data frame

In [55]:
select!(df, [1,2])

,x1,x2
,Tuple…,Tuple…
1,"(1, 1)","(1, 2)"
2,"(2, 1)","(2, 2)"
3,"(3, 1)","(3, 2)"


In [56]:
df == df2

true

Here we create a copy of `x` and delete the 3rd column from the copy with `select!` and `Not`.

In [57]:
z = copy(x)
select!(z, Not(3))

,x1,x2,x4,x5
,Tuple…,Tuple…,Tuple…,Tuple…
1,"(1, 1)","(1, 2)","(1, 4)","(1, 5)"
2,"(2, 1)","(2, 2)","(2, 4)","(2, 5)"
3,"(3, 1)","(3, 2)","(3, 4)","(3, 5)"


alternatively we can achieve the same by using the `select` function

In [58]:
select(x, Not(3))

,x1,x2,x4,x5
,Tuple…,Tuple…,Tuple…,Tuple…
1,"(1, 1)","(1, 2)","(1, 4)","(1, 5)"
2,"(2, 1)","(2, 2)","(2, 4)","(2, 5)"
3,"(3, 1)","(3, 2)","(3, 4)","(3, 5)"


`x` stays unchanged

In [59]:
x

,x1,x2,x3,x4,x5
,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…
1,"(1, 1)","(1, 2)","(1, 3)","(1, 4)","(1, 5)"
2,"(2, 1)","(2, 2)","(2, 3)","(2, 4)","(2, 5)"
3,"(3, 1)","(3, 2)","(3, 3)","(3, 4)","(3, 5)"


### Views

Note, that you can also create a view of a `DataFrame` when we want a subset of its columns:

In [60]:
@btime x[:, [1,3,5]]

  873.138 ns (22 allocations: 2.09 KiB)


,x1,x3,x5
,Tuple…,Tuple…,Tuple…
1,"(1, 1)","(1, 3)","(1, 5)"
2,"(2, 1)","(2, 3)","(2, 5)"
3,"(3, 1)","(3, 3)","(3, 5)"


In [61]:
@btime @view x[:, [1,3,5]]

  161.007 ns (4 allocations: 304 bytes)


,x1,x3,x5
,Tuple…,Tuple…,Tuple…
1,"(1, 1)","(1, 3)","(1, 5)"
2,"(2, 1)","(2, 3)","(2, 5)"
3,"(3, 1)","(3, 3)","(3, 5)"


(now creation of the `view` is slow, but in the coming releases of the DataFrames.jl package it will become significantly faster)

### Modify column by name

In [62]:
x = DataFrame([(i,j) for i in 1:3, j in 1:5])

,x1,x2,x3,x4,x5
,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…
1,"(1, 1)","(1, 2)","(1, 3)","(1, 4)","(1, 5)"
2,"(2, 1)","(2, 2)","(2, 3)","(2, 4)","(2, 5)"
3,"(3, 1)","(3, 2)","(3, 3)","(3, 4)","(3, 5)"


With the following syntax, the existing column is modified without performing any copying (this is discouraged as it creates column alias).

In [63]:
x[!, :x1] = x[!, :x2]
x

,x1,x2,x3,x4,x5
,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…
1,"(1, 2)","(1, 2)","(1, 3)","(1, 4)","(1, 5)"
2,"(2, 2)","(2, 2)","(2, 3)","(2, 4)","(2, 5)"
3,"(3, 2)","(3, 2)","(3, 3)","(3, 4)","(3, 5)"


this syntax is safer

In [64]:
x[!, :x1] = x[:, :x2]

3-element Array{Tuple{Int64,Int64},1}:
 (1, 2)
 (2, 2)
 (3, 2)

We can also use the following syntax to add a new column at the end of a `DataFrame`.

In [65]:
x[!, :A] = [1,2,3]
x

,x1,x2,x3,x4,x5,A
,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…,Int64
1,"(1, 2)","(1, 2)","(1, 3)","(1, 4)","(1, 5)",1
2,"(2, 2)","(2, 2)","(2, 3)","(2, 4)","(2, 5)",2
3,"(3, 2)","(3, 2)","(3, 3)","(3, 4)","(3, 5)",3


A new column name will be added to our `DataFrame` with the following syntax as well:

In [66]:
x.B = 11:13
x

,x1,x2,x3,x4,x5,A,B
,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…,Int64,Int64
1,"(1, 2)","(1, 2)","(1, 3)","(1, 4)","(1, 5)",1,11
2,"(2, 2)","(2, 2)","(2, 3)","(2, 4)","(2, 5)",2,12
3,"(3, 2)","(3, 2)","(3, 3)","(3, 4)","(3, 5)",3,13


### Find column name

In [67]:
x = DataFrame([(i,j) for i in 1:3, j in 1:5])

,x1,x2,x3,x4,x5
,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…
1,"(1, 1)","(1, 2)","(1, 3)","(1, 4)","(1, 5)"
2,"(2, 1)","(2, 2)","(2, 3)","(2, 4)","(2, 5)"
3,"(3, 1)","(3, 2)","(3, 3)","(3, 4)","(3, 5)"


We can check if a column with a given name exists via

In [68]:
hasproperty(x, :x1)

true

and determine its index via

In [69]:
columnindex(x, :x2)

2

### Advanced ways of column selection

these are most useful for non-standard column names (e.g. containing spaces)

In [70]:
df = DataFrame()
df.x1 = 1:3
df[!, "column 2"] = 4:6
df

,x1,column 2
,Int64,Int64
1,1,4
2,2,5
3,3,6


In [71]:
df.var"column 2"

3-element Array{Int64,1}:
 4
 5
 6

In [72]:
df[:, :var"column 2"]

3-element Array{Int64,1}:
 4
 5
 6

or you can interpolate column name using `:()` syntax

In [73]:
for n in names(df)
    println(n, "\n", df.:($n), "\n")
end

x1
[1, 2, 3]

column 2
[4, 5, 6]



### Working on a collection of columns

When using `eachcol` of a data frame the resulting object retains reference to its parent and e.g. can be queried with `getproperty`

In [74]:
df = DataFrame(reshape(1:12, 3, 4))

,x1,x2,x3,x4
,Int64,Int64,Int64,Int64
1,1,4,7,10
2,2,5,8,11
3,3,6,9,12


In [75]:
ec_df = eachcol(df)

,x1,x2,x3,x4
,Int64,Int64,Int64,Int64
1,1,4,7,10
2,2,5,8,11
3,3,6,9,12


In [76]:
ec_df[1]

3-element Array{Int64,1}:
 1
 2
 3

In [77]:
ec_df.x1

3-element Array{Int64,1}:
 1
 2
 3

### Transforming columns

We will get to this subject later in 10_transforms.ipynb notebook, but here let us just note that `select`, `select!`, `transform`, `transform!` and `combine` functions allow to generate new columns based on the old columns of a data frame.

The general rules are the following:
* `select` and `transform` always return the number of rows equal to the source data frame, while `combine` returns any number of rows (`combine` is allowed to *combine* rows of the source data frame)
* `transform` retains columns from the old data frame
* `select!` and `transform!` are in-place versions of `select` and `transform`

In [78]:
df = DataFrame(reshape(1:12, 3, 4))

,x1,x2,x3,x4
,Int64,Int64,Int64,Int64
1,1,4,7,10
2,2,5,8,11
3,3,6,9,12


Here we add a new column `:res` that is a sum of columns `:x1` and `:x2`. A general syntax of transformations of this kind is:

```
source_columns => function_to_apply => target_column_name
```
then `function_to_apply` gets columns selected by `source_columns` as positional arguments.

In [79]:
transform(df, [:x1, :x2] => (+) => :res)

,x1,x2,x3,x4,res
,Int64,Int64,Int64,Int64,Int64
1,1,4,7,10,5
2,2,5,8,11,7
3,3,6,9,12,9


One can omit passing `target_column_name` in which case it is automatically generated:

In [80]:
using Statistics

In [81]:
combine(df, [:x1, :x2] => cor)

,x1_x2_cor
,Float64
1,1.0


Note that `combine` allowed the number of columns in the resulting data frame to be changed. If we used `select` instead it would automatically broadcast the return value to match the number of rouws of the source:

In [82]:
select(df, [:x1, :x2] => cor)

,x1_x2_cor
,Float64
1,1.0
2,1.0
3,1.0


If you want to apply some function on each row of the source wrap it in `ByRow`:

In [83]:
select(df, :x1, :x2, [:x1, :x2] => ByRow(string))

,x1,x2,x1_x2_string
,Int64,Int64,String
1,1,4,14
2,2,5,25
3,3,6,36


Also if you want columns to be passed as a `NamedTuple` to a funcion (instead of being positional arguments) wrap them in `AsTable`:

In [84]:
select(df, :x1, :x2, AsTable([:x1, :x2]) => x -> x.x1 + x.x2)

,x1,x2,x1_x2_function
,Int64,Int64,Int64
1,1,4,5
2,2,5,7
3,3,6,9


For simplicity (as this functionality is often needed) there is a special treatement of `nrow` function that can be just passed as a transformation (without specifying of column selector):

In [85]:
select(df, :x1, nrow => "number_of_rows")

,x1,number_of_rows
,Int64,Int64
1,1,3
2,2,3
3,3,3


(note that in `select` the number of rows is automatically broadcasted to match the number of rows of the source data frame)